In [22]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [21]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [8]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)


The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [9]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:

# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [11]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")


Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [12]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)

    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)

    def predict(self, user, item):
        return self.forward(user, item)

In [13]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()

        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()

        #--- Producing new continuous IDs for users and movies ---

        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}

        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}

        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])


        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [14]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)


Is running on GPU: True
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0202, 0.0141, 0.0405,  ..., 0.0095, 0.0188, 0.0214],
        [0.0144, 0.0323, 0.0283,  ..., 0.0211, 0.0034, 0.0300],
        [0.0141, 0.0463, 0.0099,  ..., 0.0257, 0.0325, 0.0177],
        ...,
        [0.0385, 0.0269, 0.0120,  ..., 0.0369, 0.0376, 0.0188],
        [0.0050, 0.0309, 0.0117,  ..., 0.0384, 0.0164, 0.0332],
        [0.0162, 0.0248, 0.0113,  ..., 0.0414, 0.0215, 0.0165]])
item_factors.weight tensor([[0.0098, 0.0270, 0.0226,  ..., 0.0064, 0.0109, 0.0498],
        [0.0088, 0.0192, 0.0496,  ..., 0.0016, 0.0203, 0.0175],
        [0.0408, 0.0414, 0.0331,  ..., 0.0388, 0.0149, 0.0204],
        ...,
        [0.0266, 0.0438, 0.0364,  ..., 0.0214, 0.0139, 0.0347],
        [0.0337, 0.0273, 0.0019,  ..., 0.0045, 0.0099, 0.0074],
        [0.0192, 0.0455, 0.0441,  ..., 0.0072, 0.0383, 0.0248]])


In [15]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

<ipython-input-15-dad152416852>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: 11.059868337539246
iter #1 Loss: 4.744196672124911
iter #2 Loss: 2.4754930454764876
iter #3 Loss: 1.720301261256794
iter #4 Loss: 1.3450936247703387
iter #5 Loss: 1.1281822509874546
iter #6 Loss: 0.9909375986925841
iter #7 Loss: 0.8996656787425733
iter #8 Loss: 0.837003511674513
iter #9 Loss: 0.7918942107388816
iter #10 Loss: 0.7590866846405915
iter #11 Loss: 0.7344303795226335
iter #12 Loss: 0.7158058645383356
iter #13 Loss: 0.7013555687620555
iter #14 Loss: 0.6903314971681779
iter #15 Loss: 0.6817494792305878
iter #16 Loss: 0.6749868901626108
iter #17 Loss: 0.6694933299712723
iter #18 Loss: 0.6658182543168213
iter #19 Loss: 0.6626439871748692
iter #20 Loss: 0.6605808507216159
iter #21 Loss: 0.6588419992382151
iter #22 Loss: 0.6576280321944789
iter #23 Loss: 0.6566178359204743
iter #24 Loss: 0.6556012313907522
iter #25 Loss: 0.6545361891237612
iter #26 Loss: 0.6539151119020989
iter #27 Loss: 0.6528881548473677
iter #28 Loss: 0.6518814934011038
iter #29 Loss: 0.6506902100

In [16]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[ 1.1714,  1.2155,  1.6330,  ...,  0.8714,  0.7295,  0.9161],
        [ 0.4525,  1.4320,  1.2468,  ...,  1.2511,  1.7483,  1.0784],
        [ 1.4384,  0.7942, -0.8336,  ...,  0.8895, -0.3066,  2.4307],
        ...,
        [ 2.1541,  1.9686,  0.3405,  ...,  0.8026,  2.0062, -0.5718],
        [ 1.0348,  1.3658,  1.2085,  ...,  0.5914,  1.3074,  0.4342],
        [ 1.5803,  0.4020,  1.5250,  ...,  1.3706,  1.1648,  0.6354]],
       device='cuda:0')
item_factors.weight tensor([[0.3165, 0.4128, 0.6812,  ..., 0.3619, 0.4754, 0.7769],
        [0.0112, 0.3017, 0.5206,  ..., 1.0107, 0.1770, 0.5895],
        [0.6631, 0.4875, 0.6845,  ..., 0.5992, 0.3882, 0.0653],
        ...,
        [0.3519, 0.3520, 0.3619,  ..., 0.3464, 0.3409, 0.3596],
        [0.4290, 0.4108, 0.3968,  ..., 0.4004, 0.4064, 0.4054],
        [0.4014, 0.4161, 0.4263,  ..., 0.3910, 0.4224, 0.4068]],
       device='cuda:0')


In [17]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [18]:
len(trained_movie_embeddings) # unique movie factor weights


9724

In [19]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [20]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0
	 Ace Ventura: Pet Detective (1994)
	 Dumb & Dumber (Dumb and Dumber) (1994)
	 Babe (1995)
	 Home Alone (1990)
	 Bourne Identity, The (2002)
	 Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
	 Harry Potter and the Prisoner of Azkaban (2004)
	 Star Trek: First Contact (1996)
	 Bourne Supremacy, The (2004)
	 Maverick (1994)
Cluster #1
	 Schindler's List (1993)
	 Toy Story (1995)
	 E.T. the Extra-Terrestrial (1982)
	 Four Weddings and a Funeral (1994)
	 Casablanca (1942)
	 Wizard of Oz, The (1939)
	 Big (1988)
	 Spirited Away (Sen to Chihiro no kamikakushi) (2001)
	 When Harry Met Sally... (1989)
	 Rear Window (1954)
Cluster #2
	 Forrest Gump (1994)
	 Shawshank Redemption, The (1994)
	 Silence of the Lambs, The (1991)
	 Matrix, The (1999)
	 Jurassic Park (1993)
	 Terminator 2: Judgment Day (1991)
	 Star Wars: Episode V - The Empire Strikes Back (1980)
	 Usual Suspects, The (1995)
	 Seven (a.k.a. Se7en) (1995)
	 Raiders of the Lost 